In [6]:
import pint_pal.dmx_utils as du
from pint_pal.timingconfiguration import TimingConfiguration


config = "J2145-0750-CLEANED.chime.nb.yaml" 
par_directory = None   # default location
tim_directory = None   # default location

tc = TimingConfiguration(config, par_directory=par_directory, tim_directory=tim_directory)


using_wideband = tc.get_toa_type() == 'WB'

use_toa_pickle = False

# Use excise.tim file if it exists, else revert to raw TOAs & initial cuts
mo,to = tc.get_model_and_toas(excised=False,usepickle=use_toa_pickle)

INFO: Par file created: 2024-02-13T17:48:03.996959 [pint_pal.timingconfiguration]


2024-04-03 12:53:51.663 | DEBUG    | pint.toa:apply_clock_corrections:2200 - Applying clock corrections (include_gps = True, include_bipm = True)
2024-04-03 12:53:51.886 | INFO     | pint.observatory:gps_correction:210 - Applying GPS to UTC clock correction (~few nanoseconds)
2024-04-03 12:53:51.888 | INFO     | pint.observatory:bipm_correction:217 - Applying TT(TAI) to TT(BIPM2021) clock correction (~27 us)
2024-04-03 12:53:51.889 | INFO     | pint.observatory.topo_obs:clock_corrections:365 - Observatory chime requires no clock corrections.


KeyboardInterrupt: 

In [2]:
print(du.group_dates(to, group_width = 100.0)) # creates array of dates defining MJDs with a certain group width 
# basically splits toas into groups of width group_width

[53356.81982541 53943.54140874 54423.75335311 54641.56259376
 54767.64665249 54914.5635056  57366.93771032]


Important steps in group_dates():

In [3]:
import numpy as np
print(np.max(to.get_mjds().value)) # This does the same thing as to['mjd'] but returns it as an array -- useful
x= np.array(to.get_flag_value('be'))
print(x[0])
y = x[0] == 'GASP'
print(to[y].get_flag_value('f'))

59071.35318483562
['CHIME' 'CHIME' 'CHIME' ... 'GUPPI' 'GUPPI' 'GUPPI']
(['Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP', 'Rcvr1_2_GASP'

In [4]:
toas = to
bes = np.array(toas.get_flag_value('be')[0])  # backend flag
group_width = 0.1
 
# Only consider GASP data before the GUPPI range (PUPPI came later)
end_gasp_era = min(toas[bes == 'GUPPI'].get_mjds().value)               # --> end of GASP range is minimum GUPPI
mjds = toas.get_mjds().value
toas = toas[(mjds < end_gasp_era) & (bes == 'GASP')] # --> getting all TOAs before end of the GASP era that used the GASP backend
mjds = mjds[(mjds < end_gasp_era) & (bes == 'GASP')] # --> getting corresponding MJDs
 
# Initialize lists to be returned
low_mjds, high_mjds = [], []  # low and high MJDs (bin edges), respectively
 
# Pair up high and low frequency GASP observations                     --> do we need to separate out high & low frequency for CHIME?? if not, can we use obs?
febes = np.array(toas.get_flag_value('f')[0])  # frontend_backend flag
 
 
# these two lines just get the list of TOAs defining the new bins (for both channels/backends, since those are important for GASP), only considering gasp
low_obs_mjds = du.group_dates(toas[febes == 'Rcvr_800_GASP'], group_width) # --> group_dates "Returns MJDs of groups of TOAs no wider than a specified amount."
high_obs_mjds = du.group_dates(toas[febes == 'Rcvr1_2_GASP'], group_width)

#print(low_obs_mjds)
#print(high_obs_mjds)
# subtracts the array of mjds, which are just gasp mjds, from the newly grouped dates? don't understand how arrays are the same size
iclosest_high = [abs(mjd-high_obs_mjds).argmin() for mjd in low_obs_mjds] 
# shape is being broadcast -- mjd is a number, high_obs_mjds is an array
# subtracting array from mjd broadcasts mjd across it all
# argmin finds the index of smallest difference between the mjd being looked at and each high mjd

iaaa = [mjds for mjd in low_obs_mjds] 


print('a')
print(low_obs_mjds[0])
print(high_obs_mjds)
print(abs(low_obs_mjds[0] - high_obs_mjds))
# --> argmin() returns the *index* of the lowest vaue
iclosest_low = [abs(mjd-low_obs_mjds).argmin() for mjd in high_obs_mjds]



#print(iclosest_high)

a
53275.31548929211
[53267.18325418 53292.17590545 53347.9916923  53355.99360681
 53448.72464787 53838.40942348 53858.34509076 53889.27333475
 53918.18190948 53980.01497833 54064.78972481 54101.6793083
 54374.93208095 54429.79951559 54465.68009072 54642.2009048
 54768.86311156 54916.4449603  55094.94341097 55154.78438437
 55214.61832738]
[   8.13223512   16.86041616   72.67620301   80.67811752  173.40915858
  563.09393419  583.02960147  613.95784546  642.86642019  704.69948904
  789.47423551  826.36381901 1099.61659166 1154.4840263  1190.36460143
 1366.88541551 1493.54762227 1641.12947101 1819.62792168 1879.46889507
 1939.30283809]


In [5]:

for ihigh in enumerate(iclosest_low):
    print(f'ihigh: {ihigh}, ilow:{ilow}')

NameError: name 'ilow' is not defined

In [ ]:
bin_width = 1.0
pad = 0.0
strict_inclusion = True
check = True
toas = to

 # Order the MJDs
mjds = toas.get_mjds().value
isort = mjds.argsort()
mjds = mjds[isort] # --> just all of the TOAs in order (they aren't automatically in order)

# Initialize lists to be returned
low_mjds, high_mjds = [], []  # low and high MJDs (bin edges), respectively

# Now step through the MJDs, starting with the first (earliest)
remaining_mjds = np.copy(mjds) # --> for later
while(len(remaining_mjds)):
    left_bin_edge = remaining_mjds.min() - pad # --> logical, left bin edge is just the minimum MJD in the remaining set of MJDs

    if strict_inclusion: left_bin_edge -= 1e-11  # ~1 us # --> make the left bin edge inclusive rather than exclusive
 
    right_bin_edge = left_bin_edge + bin_width + pad # --> same logic as left edge
    
    if strict_inclusion: right_bin_edge += 1e-11  # ~1 us --> =
    print(f'Original right bin edge: {right_bin_edge}')

    in_bin = (left_bin_edge <= remaining_mjds) & \
            (remaining_mjds <= right_bin_edge) # --> create a mask that grabs only those which are in bin

    not_in_bin = np.logical_not(in_bin)  # currently not used --> okay lol

    print(remaining_mjds[in_bin])
    
    right_bin_edge = remaining_mjds[in_bin].max() + pad # shrink bin --> I guess this step makes sure everything is actually inside bin?
    print(f'Final right bin edge: {right_bin_edge}\n')
    
    if strict_inclusion: right_bin_edge += 1e-11  # ~1 us
    if np.any(in_bin):  # this should always be True
        low_mjds.append(left_bin_edge)
        high_mjds.append(right_bin_edge)
        not_accounted_for = remaining_mjds > right_bin_edge
    else:
        msg = "No TOAs in proposed DMX bin. Something is wrong."
        log.warning(msg)
    # Update remaining TOA MJDs
    remaining_mjds = remaining_mjds[not_accounted_for]